# Naive Bayes (Bayes Ingenuo) - Con tunneo de hiperparámetros


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error,r2_score


#### Carga de datos y preparación de variables


In [3]:
df = pd.read_csv("train.csv")

# Variables predictoras (las mismas que en modelos anteriores)
variables_numericas = [
    'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF',
    '1stFlrSF', 'FullBath', 'YearBuilt', 'KitchenAbvGr',
    'TotRmsAbvGrd', 'Fireplaces', 'SalePrice'
]

df = df[variables_numericas]

#### Definir X (variables predictoras) y "y" (variable respuesta)


In [4]:
X = df.drop(['SalePrice'], axis=1)
y = df['SalePrice']

#### Dividir datos en entrenamienta y prueba


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.7, test_size=0.3)

#### Definir hiperparámetros para el modelo y su pipeline


In [6]:

param_grid_naive_bayes =  [{
    "regressor__var_smoothing": np.logspace(0,-9, num=100)
}]

numerical_preprocessor = StandardScaler()
preprocessor = ColumnTransformer([
    ('num', numerical_preprocessor, X.columns)
])

naive_bayes_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', GaussianNB())
])

#### Naïve Bayes


In [7]:
bayes_regressor = GridSearchCV(naive_bayes_pipeline,param_grid=param_grid_naive_bayes,n_jobs=2, cv=5,scoring="neg_root_mean_squared_error")
bayes_regressor.fit(X_train, y_train)
model = bayes_regressor.best_estimator_
print(model)
print('best score:')
print(bayes_regressor.best_score_)
y_mejor_pred = bayes_regressor.best_estimator_.predict(X_test)

c:\Users\ncast\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', '1stFlrSF',
       'FullBath', 'YearBuilt', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces'],
      dtype='object'))])),
                ('regressor', GaussianNB(var_smoothing=1.0))])
best score:
-39670.409417066854


#### Evaluación de Modelo en Conjunto de Prueba


In [10]:
y_pred = model.predict(X_test)
mae_nb = mean_absolute_error(y_test, y_mejor_pred)
mse_nb = mean_squared_error(y_test, y_mejor_pred)
rmse_nb = np.sqrt(mse_nb)

print(f"Mean Absolute Error (MAE): {mae_nb:.2f}")
print(f"Mean Squared Error (MSE): {mse_nb:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse_nb:.2f}")

Mean Absolute Error (MAE): 26539.12
Mean Squared Error (MSE): 1548788039.38
Root Mean Squared Error (RMSE): 39354.64


<small>

### **Conclusión: Modelo de Regresión (Naïve Bayes)**

Después del ajuste de hiperparámetros, el **modelo de regresión basado en Naïve Bayes** mostró una **mejora significativa en todas las métricas** de error:

- **MAE** (Error Absoluto Medio) se redujo de **30,461.95** a **26,527.59**, lo que indica que las predicciones son en promedio **casi 4,000 unidades más precisas**.
- **MSE** (Error Cuadrático Medio) disminuyó de **2,119,162,831.51** a **1,548,788,039.38**, lo que significa que el modelo produce errores menos extremos.
- **RMSE** (Raíz del Error Cuadrático Medio) se redujo de **46,034.37** a **39,354.64**, lo que confirma que los errores de predicción se han reducido en términos generales.

**Esto indica que el modelo optimizado tiene una mejor capacidad de generalización y proporciona predicciones más precisas.** La reducción en el RMSE es especialmente importante, ya que refleja que el modelo ahora comete errores **menos severos** al predecir el precio de las casas.

### **Conclusión Global**

El proceso de **ajuste de hiperparámetros** logró **mejorar ambos modelos**.

- En **regresión**, el modelo ahora comete **errores menos graves y predice valores más cercanos a los reales**.
- En **clasificación**, el modelo ahora **identifica mejor las clases y reduce la cantidad de errores de categorización**.

<small>
